In [1]:
import arcpy
from datetime import datetime
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# path output folder
outputs = '.\\Outputs'

In [6]:
# load data
hui = r"E:\Projects\Housing-Unit-Inventory\Outputs\wfrc_housing_unit_inventory_20210910.gdb\wfrc_housing_unit_inventory"
hui_sdf = pd.DataFrame.spatial.from_featureclass(hui)
hui_sdf.columns

Index(['OBJECTID', 'PARCEL_ID', 'TYPE_WFRC', 'SUBTYPE_WFRC', 'NOTE', 'IS_OUG',
       'CITY', 'SUBREGION', 'COUNTY', 'HOUSE_CNT', 'ADDR_CNT', 'UNIT_COUNT',
       'PARCEL_COUNT', 'FLAG', 'DUA', 'FLOORS_CNT', 'PARCEL_ACRES',
       'BLDG_SQFT', 'TOTAL_MKT_VALUE', 'BUILT_YR', 'BUILT_DECADE', 'UNIT_ID',
       'ADJUSTED', 'REVIEWED', 'SHAPE'],
      dtype='object')

In [14]:
hui_sdf.loc[(hui_sdf['REVIEWED'] != 1) & (hui_sdf['IS_OUG'] != 1) & (hui_sdf['ADDR_CNT'] > 3), 
            'UNIT_COUNT'] = hui_sdf['ADDR_CNT']

In [17]:
# create output gdb
date = datetime.today().strftime('%Y%m%d')
gdb = os.path.join(outputs, "wfrc_housing_unit_inventory_{}.gdb".format(date))
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "wfrc_housing_unit_inventory_{}.gdb".format(date))
    
hui_sdf.spatial.to_featureclass(location=os.path.join(gdb, 'wfrc_housing_unit_inventory'),sanitize_columns=False)

'E:\\Projects\\Housing-Unit-Inventory\\Outputs\\wfrc_housing_unit_inventory_20210914.gdb\\wfrc_housing_unit_inventory'

In [18]:
# Export
hui_sdf_no_shape = hui_sdf.copy()
del hui_sdf_no_shape['SHAPE']
hui_sdf_no_shape.to_csv(os.path.join(outputs, 'wfrc_housing_unit_inventory_{}.csv'.format(date)), index=False)